# Image Segmentation using U-Net implemented in Tensorflow

In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import tensorflow.keras as keras
from keras import layers
import os
import matplotlib.pyplot as plt
from glob import glob
from os.path import split, splitext, join

%matplotlib inline
import numpy as np
import skimage.transform
import matplotlib as mpl
import matplotlib.pyplot as plt # plotting
# from skimage.io import imread # read in images
# from skimage.segmentation import mark_boundaries # mark labels
from sklearn.metrics import roc_curve, auc # roc curve tools
from skimage.color import label2rgb
import numpy as np # linear algebra / matrices
import pandas as pd

from IPython.display import display
# from skimage.util.montage import montage2d



In [ ]:
# # copy the weights and configurations for the pre-trained models (before any imports)
# !rm -rf ~/.keras
# !mkdir ~/.keras
# !mkdir ~/.keras/models
# !cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
# !cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/

In [52]:
# Establish filepaths relative to the kaggle working directory
%cd ~/../kaggle/working
BASEDIR = "../input/carvana-image-masking-challenge"
metadata = pd.read_csv(join(BASEDIR, 'metadata.csv.zip'))
TRAIN_PATH = os.path.join(BASEDIR, 'train.zip')
TEST_PATH = os.path.join(BASEDIR, 'test.zip')

/kaggle/working


In [58]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# extract a list of files names
files = os.listdir(os.path.join(BASEDIR))

/kaggle/input/carvana-image-masking-challenge/train_masks.zip
/kaggle/input/carvana-image-masking-challenge/29bb3ece3180_11.jpg
/kaggle/input/carvana-image-masking-challenge/train_masks.csv.zip
/kaggle/input/carvana-image-masking-challenge/train.zip
/kaggle/input/carvana-image-masking-challenge/metadata.csv.zip
/kaggle/input/carvana-image-masking-challenge/sample_submission.csv.zip
/kaggle/input/carvana-image-masking-challenge/test.zip
/kaggle/input/carvana-image-masking-challenge/test_hq.zip
/kaggle/input/carvana-image-masking-challenge/train_hq.zip


In [54]:
# report the shape ofthe metadata file
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6572 entries, 0 to 6571
Data columns (total 6 columns):
id       6572 non-null object
year     6561 non-null float64
make     6561 non-null object
model    6561 non-null object
trim1    6510 non-null object
trim2    6495 non-null object
dtypes: float64(1), object(5)
memory usage: 308.2+ KB


Always a good idea to start off by examining the metadata. In this case each id corresponds to a unique car, and there are 16 photos of each car containined in the training and test data, in JPEG format.


In [55]:
train_df = pd.DataFrame(dict(path = glob(TRAIN_PATH)))ls -la

train_df

,path
0,../input/carvana-image-masking-challenge/train...


In [ ]:
os.path.join(BASEDIR + 'train.zip', '*.*')

In [ ]:
# extract training and test data
# import zipfile

# datasets = ["train", "test"]

# for dataset in datasets:
#     with zipfile.ZipFile(BASEDIR + '/' + dataset +'.zip',"r") as z:
#         z.extractall("./working/" + dataset)



In [94]:
all_img_df = pd.DataFrame(dict(path = glob(join('working/train/train', '*.*'))))
all_img_df['key_id'] = all_img_df['path'].map(lambda x: splitext(os.path.basename(x))[0])
all_img_df['car_id'] = all_img_df['key_id'].map(lambda x: x.split('_')[0])
all_img_df['mask_path'] = all_img_df['path'].map(lambda x: x.replace('train', 'train_masks').replace('.jpg', '_mask.gif'))
all_img_df['exists'] = all_img_df['mask_path'].map(os.path.exists)
print(all_img_df['exists'].value_counts())
all_img_df

False    5088
Name: exists, dtype: int64


,path,key_id,car_id,mask_path,exists
0,working/train/train/bad53871c7d9_12.jpg,bad53871c7d9_12,bad53871c7d9,working/train_masks/train_masks/bad53871c7d9_1...,False
1,working/train/train/dd47eb7ac4ee_10.jpg,dd47eb7ac4ee_10,dd47eb7ac4ee,working/train_masks/train_masks/dd47eb7ac4ee_1...,False
2,working/train/train/3c6acfceb552_11.jpg,3c6acfceb552_11,3c6acfceb552,working/train_masks/train_masks/3c6acfceb552_1...,False
3,working/train/train/bec93f7ddb19_08.jpg,bec93f7ddb19_08,bec93f7ddb19,working/train_masks/train_masks/bec93f7ddb19_0...,False
4,working/train/train/3627b6f4c16d_13.jpg,3627b6f4c16d_13,3627b6f4c16d,working/train_masks/train_masks/3627b6f4c16d_1...,False
...,...,...,...,...,...
5083,working/train/train/0cdf5b5d0ce1_02.jpg,0cdf5b5d0ce1_02,0cdf5b5d0ce1,working/train_masks/train_masks/0cdf5b5d0ce1_0...,False
5084,working/train/train/0495dcf27283_11.jpg,0495dcf27283_11,0495dcf27283,working/train_masks/train_masks/0495dcf27283_1...,False
5085,working/train/train/a56f923399ca_08.jpg,a56f923399ca_08,a56f923399ca,working/train_masks/train_masks/a56f923399ca_0...,False
5086,working/train/train/8b88fb784951_12.jpg,8b88fb784951_12,8b88fb784951,working/train_masks/train_masks/8b88fb784951_1...,False


In [ ]:
def read_diff_img(c_row):
    t0_img = imread(c_row['path'])[:,:,0:3]
    cg_img = imread(c_row['mask_path'], as_grey=True)
    return (t0_img, cg_img)

def make_change_figure(c_row):
    a,c = read_diff_img(c_row)
    fig, (ax1, ax3) = plt.subplots(1, 2, figsize = (21,7))
    ax1.imshow(a)
    ax1.set_title('Before')
    ax1.axis('off')
    d = skimage.measure.label(c)
    ax3.imshow(d, cmap = 'nipy_spectral_r')
    ax3.set_title('Changes')
    ax3.axis('off')
    return fig
_, t_row = next(all_img_df.sample(1).iterrows())
make_change_figure(t_row).savefig('overview.png', dpi = 300)
a,c = read_diff_img(t_row)
print(a.shape, c.shape)

In [ ]:
from sklearn.model_selection import train_test_split
def train_test_split_on_group(in_df, col_id, **kwargs):
    group_val = np.unique(in_df[col_id])
    train_ids, test_ids = train_test_split(group_val, **kwargs)
    return in_df[in_df[col_id].isin(train_ids)], in_df[in_df[col_id].isin(test_ids)],
train_df, valid_df = train_test_split_on_group(all_img_df, col_id = 'car_id', random_state = 2018, test_size = 0.2)
valid_df, test_df = train_test_split_on_group(valid_df, col_id = 'car_id', random_state = 2018, test_size = 0.5)
print(train_df.shape[0], 'training images')
print(valid_df.shape[0], 'validation images')
print(test_df.shape[0], 'test images')